# Gabor Dataset with Random Forest

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import operator
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [50]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'product_data.pkl')
product_data

,order_id,orderdate,total_order_price,customer__id,product__id,productName,amount,totalPrice,productSizeEu,o_parentId,...,indicatorSale,weight__value,color,productColorName,productType,materialPm,material,line,originCountry,gender
0,2516727,2020-01-01 10:08:54,89.95,2188549.0,557559,sportliche Ballerinas Glattleder schwarz,1,89.95,40.0,337523,...,0,0.71,0,0,0,0,0,0,0,0
1,2587743,2020-02-18 16:57:17,269.85,2319469.0,557559,sportliche Ballerinas Glattleder schwarz,1,89.95,40.0,337523,...,0,0.71,0,0,0,0,0,0,0,0
2,4328335,2020-10-10 05:42:10,146.16,4328338.0,557559,Sportliche Ballerina Glattleder schwarz,1,87.68,40.0,337523,...,0,0.71,0,0,0,0,0,0,0,0
3,7706849,2021-12-25 21:10:51,188.95,7706853.0,557559,Sportliche Ballerina Glattleder schwarz,1,89.95,40.0,337523,...,0,0.71,0,0,0,0,0,0,0,0
4,7707311,2021-12-26 08:32:26,89.95,2103820.0,557559,Sportliche Ballerina Glattleder schwarz,1,89.95,40.0,337523,...,0,0.71,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648487,10896317,2023-02-27 15:21:15,360.00,10896322.0,10225300,PG1137 Sneaker low Materialmix Leder/Lederimit...,1,110.00,49.5,10223435,...,1,1.05,7,16,0,1098,5,2,4,1
648488,10941648,2023-03-04 18:15:57,220.00,2013733.0,10225288,PG1137 Sneaker low Materialmix Leder/Lederimit...,1,110.00,40.5,10223435,...,1,1.05,7,16,0,1098,5,2,4,1
648489,11015232,2023-03-12 17:54:41,390.00,11015227.0,10225287,PG1137 Sneaker low Materialmix Leder/Lederimit...,1,110.00,40.0,10223435,...,1,1.05,7,16,0,1098,5,2,4,1
648490,10898392,2023-02-27 20:23:50,245.00,10898300.0,10225292,PG1137 Lage sneaker Combi leer/Imitatieleer grijs,1,115.00,43.0,10223435,...,1,1.05,7,18,0,1098,5,2,4,1


In [51]:
#product_data.head(3).to_csv('test.csv', index=False)

In [52]:
df_recsys = product_data.drop(['order_id', 'orderdate', 'total_order_price', 'customer__id', 'productName', 'totalPrice', 'productSizeEu', 'o_parentId', 'brand__id', 'mainCategory__id', 'name', 'b2cPrice', 'indicatorAdvertisement', 'indicatorWedding', 'indicatorSale', 'weight__value', 'color', 'productColorName', 'productType', 'materialPm', 'material', 'line', 'originCountry', 'gender'], axis=1)
df_recsys = df_recsys.groupby('product__id').agg({'amount': 'sum'})
df_recsys.reset_index(inplace=True)
df_recsys['product__id'] = pd.Categorical(df_recsys['product__id'])

#df_recsys = df_recsys.sample(n=10000)
df_recsys

,product__id,amount
0,360824,10
1,360825,31
2,360826,68
3,360827,85
4,360828,54
...,...,...
108772,11032275,1
108773,11032282,2
108774,11032283,1
108775,11032287,1


In [53]:
# split the data into training and testing sets
X = pd.get_dummies(df_recsys['product__id'], prefix='product')
y = df_recsys['amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
# train the random forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10, verbose=True, n_jobs=-1)
rf.fit(X_train, y_train)



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 35.6min finished


RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=42, verbose=True)

In [55]:
# evaluate the model
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE: ", mse)
print("MAE: ", mae)
print("R-squared: ", r2)

MSE:  89.06732744696097
MAE:  4.956987306143963
R-squared:  -0.00035567573464057034


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.0s finished



### 1000 items
MSE:  90.897
MAE:  4.91
R-squared:  -0.3609596804259385

### 10.000 items

MSE:  84.581
MAE:  4.86
R-squared:  -0.38745173175156755


### all data
36 min training
MSE:  89.06732744696097
MAE:  4.956987306143963
R-squared:  -0.00035567573464057034


In [60]:

from datetime import datetime
from joblib import dump

now = datetime.now()
date_string = now.strftime("%d.%m.%Y %H:%M")

filename = '../../models/gabor/random_forest_classifier_model-'+ date_string +'.joblib'
dump(rf, filename)



['../../models/gabor/random_forest_classifier_model-27.04.2023 09:02.joblib']

In [62]:
from joblib import load

# load the saved model
model = load(filename)

NameError: name 'filename' is not defined